# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time
# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#Here I am importing the data I collected from the Weather exercise
data = pd.read_csv("cities.csv")
#Renaming the first column to "City"
data = data.rename(columns={"Unnamed: 0":"City"})

#Filtering out 'NaN' data from Lat and Lon
filter_data = data[data[['Lat', 'Lng']].notnull().all(1)]
filter_data.head()

,City,Cloudiness,Country,Humidity,Lat,Lng,Max Temp,Wind Speed
0,rio grande,90.0,BR,83.0,-32.03,-52.10,75.2,14.99
1,punta arenas,75.0,CL,58.0,-53.15,-70.92,59.0,21.92
2,carnarvon,75.0,AU,78.0,-24.87,113.63,77.0,8.05
3,arraial do cabo,40.0,BR,83.0,-22.97,-42.03,80.6,11.41
4,tuktoyaktuk,90.0,CA,79.0,69.45,-133.04,23.0,28.86


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#Configure Google Maps
gmaps.configure(api_key=g_key)

#Storing my lat&lon data and humidity data
location = filter_data[["Lat","Lng"]]
humidity = filter_data['Humidity'].astype(float)

#Plot heat map
fig = gmaps.figure()

#Creating a heat map layer
heatMap = gmaps.heatmap_layer(location, weights = humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=2)
#Adding heat layer to 
fig.add_layer(heatMap)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
#Filtering down to ideal weather spots
idealWeather = filter_data.loc[filter_data["Cloudiness"]<=15]
idealWeather = idealWeather.loc[filter_data["Max Temp"]>=65]
idealWeather = idealWeather.loc[filter_data["Max Temp"]<=95]
idealWeather = idealWeather.loc[filter_data["Wind Speed"]<=20]
hotel_df = idealWeather

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
#showing the current data frame
hotel_df.head()


,City,Cloudiness,Country,Humidity,Lat,Lng,Max Temp,Wind Speed
10,rikitea,0.0,PF,80.0,-23.12,-134.97,77.67,13.69
13,hobyo,10.0,SO,79.0,5.35,48.53,80.01,14.94
16,guatire,0.0,VE,63.0,10.48,-66.54,77.00,9.17
22,castro,0.0,BR,59.0,-24.79,-50.01,73.17,2.06
25,atuona,12.0,PF,78.0,-9.80,-139.03,81.45,12.86


In [6]:
#Resetting the index because the numbers got mixed up when filtering data
hotel_df.reset_index(inplace = True)
hotel_df.head()

,index,City,Cloudiness,Country,Humidity,Lat,Lng,Max Temp,Wind Speed
0,10,rikitea,0.0,PF,80.0,-23.12,-134.97,77.67,13.69
1,13,hobyo,10.0,SO,79.0,5.35,48.53,80.01,14.94
2,16,guatire,0.0,VE,63.0,10.48,-66.54,77.00,9.17
3,22,castro,0.0,BR,59.0,-24.79,-50.01,73.17,2.06
4,25,atuona,12.0,PF,78.0,-9.80,-139.03,81.45,12.86


In [7]:
#Inserting a new column called 'Hotel Name'

hotel_df['Hotel Name'] = ""
hotel_df.head()

,index,City,Cloudiness,Country,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,10,rikitea,0.0,PF,80.0,-23.12,-134.97,77.67,13.69,
1,13,hobyo,10.0,SO,79.0,5.35,48.53,80.01,14.94,
2,16,guatire,0.0,VE,63.0,10.48,-66.54,77.00,9.17,
3,22,castro,0.0,BR,59.0,-24.79,-50.01,73.17,2.06,
4,25,atuona,12.0,PF,78.0,-9.80,-139.03,81.45,12.86,


In [8]:
#Setting up the base URL and the search parameters
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
target_search = "hotel"
target_radius = 5000

#This loop below takes the lat and lng one at a time, plugs them into 
#the search, finds the first hotel, then stores it into the data base
x=0
for index,row in hotel_df.iterrows():
    #Setting up the different lat and lngs
    lat = hotel_df["Lat"][x]
    lng = hotel_df["Lng"][x]
    target_coordinates = (f'{lat}, {lng}')    
    
    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
        "key": g_key
    }

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)

    #Running request for the given parameters
    print(target_coordinates)
    response = requests.get(base_url, params=params).json()
    
    #Iterates to the next index in the dataframe
    x=x+1
    
    #Store the hotel name into the dataframe
    try:
        hotel_df.loc[index, 'Hotel Name'] = response["results"][0]['name']
    except:
        hotel_df.loc[index, 'Hotel Name'] = ''
        
    

-23.12, -134.97
5.35, 48.53
10.48, -66.54
-24.79, -50.01
-9.8, -139.03
12.18, 18.69
-7.95, 110.28
-34.62, -68.33
-18.51, -54.76
27.98, -114.06
-26.78, -51.02
8.46, 25.68
-16.48, -151.75
27.04, 14.43
20.7, -105.2
18.69, 12.92
-25.24, 130.99
13.02, 31.87
-23.86, 35.38
5.95, 80.54
-38.55, -58.74
22.08, -159.32
-10.41, -36.43
-15.77, 128.73
-24.49, -47.84
23.3, 113.83
-25.4, -70.48
19.93, -105.25
-26.65, 15.16
-26.62, -54.13
32.0, -102.08
-29.04, 24.6
29.54, 67.88
9.41, 45.06
18.74, 7.39
12.17, -14.75
-38.95, -68.06
4.95, -77.37
-28.45, 18.73
13.64, 16.49
14.73, 0.92
-35.1, 173.78
20.5, -10.07
11.0, 122.67
-23.78, -45.36
10.57, 72.64
24.09, 32.91
20.71, -78.0
10.88, 122.49
31.42, -103.49
26.54, -77.06
27.02, -108.93
-26.5, 17.16
-28.78, 32.04
-4.97, 39.83
34.26, -88.7
14.1, -1.63
30.76, -86.57
-23.13, 150.73
-16.78, -151.5


In [9]:
hotel_df.head()


,index,City,Cloudiness,Country,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,10,rikitea,0.0,PF,80.0,-23.12,-134.97,77.67,13.69,Pension Maro'i
1,13,hobyo,10.0,SO,79.0,5.35,48.53,80.01,14.94,
2,16,guatire,0.0,VE,63.0,10.48,-66.54,77.00,9.17,Grand Hotel
3,22,castro,0.0,BR,59.0,-24.79,-50.01,73.17,2.06,Pousada Rota Dos Tropeiros
4,25,atuona,12.0,PF,78.0,-9.80,-139.03,81.45,12.86,Simplicité Marquises


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map

# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations)
# Add the layer to the map
fig.add_layer(markers)
fig

# Display Map

Figure(layout=FigureLayout(height='420px'))